In [3]:
import numpy as np

In [4]:
# 计算下中位数
def getDownMedian(data):
    if data.shape[0] % 2 == 0:
        data = np.hstack([data, np.inf])
    return np.median(data)
    
def seperateNode(node, depth):
    dataSet = node['data']
    if dataSet.shape[0] == 0:
        return None
    feature = depth % dataSet.shape[1]
    value = getDownMedian(dataSet[:, feature])
    node['left'] = {'data':dataSet[dataSet[:,feature] < value, :],
                    'left':None, 'right':None, 'parent':node}
    node['right'] = {'data':dataSet[dataSet[:,feature] > value, :],
                     'left':None, 'right':None, 'parent':node}
    node['data'] = dataSet[dataSet[:,feature] == value, :][0]
    node['feature'] = feature
    seperateNode(node['left'], depth+1)
    seperateNode(node['right'], depth+1)
    return node
    
def printNode(node, depth):
    if node == None or node['data'].shape[0]==0:
        return
    print (" "*depth, node['data'])
    printNode(node['left'], depth+1)
    printNode(node['right'], depth+1)
    
    
def createTree(dataSet):
    root = {'data':dataSet, 'left':None, 'right':None}
    seperateNode(root, 0)
    return root 

In [5]:
root = createTree(np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]))
printNode(root, 0)

 [7 2]
  [5 4]
   [2 3]
   [4 7]
  [9 6]
   [8 1]


In [50]:
def isLeaf(node):
    return True

# 在kd树中找出包含目标点x的叶结点
def findLeaf(node, depth, x):
    # 从根结点出发，递归地向下访问kd树
    while True:
        feature = node['feature']
        # 若目标点x当前维的坐标小于切分点的坐标
        if x[feature] < node['data'][feature]:
            # 则移动到左子节点
            subnode = node['left']
        # 否则移动到右子节点
        else:
            subnode = node['right']
        # 直至子结点为叶结点为止
        if subnode['data'].shape[0] == 0:
            return node
        node = subnode

def distance(A, B):
    return np.linalg.norm(A - B)

def check(feature, value, x):
    return np.abs(x[feature] - value[feature])

def isSame(a, b):
    return a[0] == b[0] and a[1] == b[1]

def findAnother(node):
    parent = node['parent']
    if isSame(node['data'], parent['left']['data']):
        return parent['right']
    else:
        return parent['left']
        
def search(root, x):
    # 在kd树中找出包含目标点x的叶结点
    current = findLeaf(root, 0, x)
    # 以此结点为当前最近点
    nearest = current['data']
    currentDistance = distance(nearest, x)
    # 递归地向上回退，在每个结点进行以下操作
    # 当回退到根结点时搜索结束
    while not isSame(current['data'], root['data']):
        parent = current['parent']
        dis = distance(parent['data'], x)
        # 如果该结点保存的实例点比当前最近点距离目标点更近
        if dis < currentDistance:
            # 则以该实例点为“当前最近点”
            nearest = parent['data']
            currentDistance = dis
        # 检查该子结点的父结点的另一个子结点对应的区域是否有更近的点
        # 即目标结点到平面feature=value的距离
        # 如果相交，可能在另一个子结点对应的区域内存在距目标点更近的点
        if check(parent['feature'], parent['data'], x) < currentDistance:
            # 递归的进行最近邻搜索
            newnode, dis = search(findAnother(current), x)
            if dis < currentDistance:
                nearest = newnode
                currentDistance = dis
        current = parent  
    return nearest, currentDistance
        

In [51]:
root = createTree(np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]))
search(root, np.array([8, 7]))

(array([9, 6]), 1.4142135623730951)